In [1]:
from data_transformation import utils
from definitions.file_paths import CSV_RS_CASES, CSV_LI_CASES, CSV_CASE_CITATIONS, CSV_LEGISLATION_CITATIONS
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
from pprint import pprint
ECLI = 'ecli'

/home/maxine/anaconda3/envs/case-law-explorer/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


# Inspection and mapping of Rechtspraak (RS) to Legal Intelligence (LI) + citation (LIDO) data

------------------------------------------------------------------------------------------------------------------------
### DATES
#### Raw dates

In [ ]:
dates_rs = ['issued', 'date']
dates_li = ['PublicationDate', 'EnactmentDate', 'DateAdded']
df_dates_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + dates_rs).rename(columns={'identifier': ECLI})
df_dates_li = utils.read_csv(CSV_LI_CASES, [ECLI] + dates_li)
df_dates = utils.map_cases(df_dates_rs.copy(), df_dates_li.copy())

utils.not_none_rows(df_dates, number=2)

#### Clean dates
(convert dates to datetime)

In [ ]:
df_dates_clean = df_dates.apply(lambda col: col.apply(
    lambda x: utils.format_li_date(x) if x else x) if col.name in dates_li else col)

df_dates_clean.describe()

#### Check if RS issued always before LI PublicationDate:

In [ ]:
utils.compare('issued', 'PublicationDate', '<=', df_dates)

#### Check for equality of RS date and LI EnactmentDate

In [ ]:
utils.compare('date', 'EnactmentDate', '==', df_dates)

#### Check if LI EnactmentDate always before LI PublicationDate

In [ ]:
utils.compare('EnactmentDate', 'PublicationDate', '<=', df_dates)

#### Check if LI PublicationDate always before LI DateAdded

In [ ]:
utils.compare('PublicationDate', 'DateAdded', '<=', df_dates)

#### Summary dates:

original format LI dates: YYYYMMDD (or YYYYMMDD.0 if parsed as float)<br>
original format RS dates: YYYY-MM-DD

- RS date corresponds to LI EnactmentDate --> date of decision
- RS issued before LI PublicationDate --> date published online (on RS/ other publisher)
- LI EnactmentDate before LI PublicationDate
- PublicationDate before DateAdded --> DateAdded = date added to LI platform

In [ ]:
del dates_rs, dates_li, df_dates, df_dates_li, df_dates_rs, df_dates_clean

------------------------------------------------------------------------------------------------------------------------
### TEXTUAL INFORMATION
#### Raw texts

In [ ]:
texts_rs = ['inhoudsindicatie', 'info']
texts_li = ['Summary']
df_texts_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + texts_rs).rename(columns={'identifier': ECLI})
df_texts_li = utils.read_csv(CSV_LI_CASES, [ECLI] + texts_li)
df_texts = utils.map_cases(df_texts_rs.copy(), df_texts_li.copy())

utils.not_none_rows(df_texts, number=1)

#### Clean texts
(remove xml tags)

In [ ]:
df_texts_clean = df_texts.apply(lambda col: col.apply(
    lambda x: utils.format_rs_xml(x) if x else x) if col.name in texts_rs else col)

df_texts_clean.describe()

In [ ]:
utils.not_none_rows(df_texts_clean, number=5)

#### Summary text information:

original format RS texts: XML <br>
original format LI texts: plain string

- LI Summary: contains a short summary of the case in natural language
- RS inhoudsindicatie: contains keywords (legislation articles) / short summary of case
 (--> RS inhoudsindicatie ~ LI Summary (<- more informative))
- RS info: contains meta info about case (court, case number, date, type, attorney, ...)

In [ ]:
del df_texts_clean, df_texts_li, df_texts_rs, df_texts, texts_rs, texts_li


------------------------------------------------------------------------------------------------------------------------
### TITLES
#### Raw titles

In [ ]:
titles_rs = ['title']
titles_li = ['Title', 'DisplayTitle', 'DisplaySubtitle']

df_titles_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + titles_rs).rename(columns={'identifier': ECLI})
df_titles_li = utils.read_csv(CSV_LI_CASES, [ECLI] + titles_li)
df_titles = utils.map_cases(df_titles_rs.copy(), df_titles_li.copy())

utils.not_none_rows(df_titles, number=2)

#### Clean titles
(remove white spaces before and after text)

In [ ]:
df_titles_clean = df_titles.copy()
df_titles_clean = df_titles_clean.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))

df_titles_clean.describe()

In [ ]:
utils.not_none_rows(df_titles_clean, number=5)

In [ ]:
utils.compare('Title', 'DisplaySubtitle', '==', df_titles_clean[titles_li])

Summary titles:

original format RS titles: plain string<br>
original format LI titles: plain string

- LI Title == LI DisplaySubtitle: concise description of case
- RS title ~ LI DisplayTitle: meta info about case (case number, court, date, ...)

In [ ]:
del df_titles, df_titles_clean, df_titles_li, df_titles_rs, titles_li, titles_rs

------------------------------------------------------------------------------------------------------------------------
### CASE NUMBERS
#### Raw case numbers

In [ ]:
case_numbers_rs = ['zaaknummer']
case_numbers_li = ['Id', 'CaseNumber', 'PublicationNumber', 'IssueNumber']

df_case_numbers_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + case_numbers_rs).rename(columns={'identifier': ECLI})
df_case_numbers_li = utils.read_csv(CSV_LI_CASES, [ECLI] + case_numbers_li)
df_case_numbers = utils.map_cases(df_case_numbers_rs.copy(), df_case_numbers_li.copy())

utils.not_none_rows(df_case_numbers, number=2)

#### Clean case numbers
(remove white spaces before and after text)

In [ ]:
df_case_numbers_clean = df_case_numbers.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))

df_case_numbers_clean.describe()

In [ ]:
utils.not_none_rows(df_case_numbers_clean, number=5)


#### Summary case numbers:

original format RS case numbers: plain string<br>
original format LI case numbers: plain string

- RS ecli (identifier): ecli
- RS zaaknummer: identifier before eclis introduced
- LI Id: internal entry ID within LI
- LI CaseNumber: combination of ecli and zaaknummer
- LI PublicationNumber: document ID of publishing institution
- LI IssueNumber: collection ID of publishing institution

In [ ]:
del case_numbers_li, case_numbers_rs, df_case_numbers_clean, df_case_numbers_li, df_case_numbers_rs, df_case_numbers

------------------------------------------------------------------------------------------------------------------------
### DOMAINS
#### Raw domains

In [ ]:
domains_rs = ['subject']
domains_li = ['LawArea']

df_domains_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + domains_rs).rename(columns={'identifier': ECLI})
df_domains_li = utils.read_csv(CSV_LI_CASES, [ECLI] + domains_li)

df_domains = utils.map_cases(df_domains_rs.copy(), df_domains_li.copy())

utils.not_none_rows(df_domains, number=2)

#### Clean domains
(remove white spaces before and after text, transform list notation to "e1; e2; e3")

In [ ]:
df_domains_clean = df_domains.copy()
df_domains_clean = df_domains_clean.apply(lambda col: col.apply(
    lambda x: utils.format_rs_list(x) if x else x) if col.name in domains_rs else col)
df_domains_clean = df_domains_clean.apply(lambda col: col.apply(
    lambda x: utils.format_li_list(x) if x else x) if col.name in domains_li else col)

df_domains_clean.describe()

In [ ]:
# find unqiue domain names of RS and LI:
domains_rs_list = df_domains_rs['subject'].value_counts().index.tolist()
unique_domains_rs = list(set(sum([i.split('; ') for i in domains_rs_list], [])))
unique_domains_rs.sort()
print("Unique RS domains:")
pprint(unique_domains_rs)

df_domains_li_clean = df_domains_li.apply(lambda col: col.apply(utils.format_li_list))
domains_li_list = df_domains_li_clean['LawArea'].value_counts().index.tolist()
unique_domains_li = list(set(sum([i.split('; ') for i in domains_li_list], [])))
unique_domains_li.sort()
print("\nUnique LI domains:")
pprint(unique_domains_li)

In [ ]:
utils.not_none_rows(df_domains_clean, number=5)

#### Summary domains:

original format RS domains: "d1; d2; d3"<br>
original format LI domains: "['d1', 'd2', 'd3']"

- RS and LI use different notation to label case domains
- RS and LI domains do not always match

In [ ]:
del domains_rs, domains_li, df_domains_li, df_domains_rs, df_domains_clean, df_domains

------------------------------------------------------------------------------------------------------------------------
### INSTANCE
#### Raw instance

In [ ]:
instance_rs = ['creator']
instance_li = ['IssuingInstitution']

df_instance_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + instance_rs).rename(columns={'identifier': ECLI})
df_instance_li = utils.read_csv(CSV_LI_CASES, [ECLI] + instance_li)
df_instance = utils.map_cases(df_instance_rs.copy(), df_instance_li.copy())

utils.not_none_rows(df_instance, number=2)

#### Clean instance
(remove white spaces before and after text)

In [ ]:
df_instance_clean = df_instance.copy()
df_instance_clean = df_instance_clean.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))
df_instance_clean.describe()

In [ ]:
utils.compare('creator', 'IssuingInstitution', '==', df_instance, number=37)

#### Summary instance:

- RS uses "'s-Gravenhage", "'s Gravenhage" and "Den Haag" to refer to Den Haag
- LI uses only "Den Haag" to refer to Den Haag
- RS "Rechtbank" == LI "Sector kanton Rechtbank")

In [ ]:
del instance_li, instance_rs, df_instance_li, df_instance_clean, df_instance, df_instance_rs

------------------------------------------------------------------------------------------------------------------------
### JURISDICTION
#### Raw jurisdiction

In [ ]:
jurisdiction_rs = ['spatial', 'jurisdiction_country']
jurisdiction_li = ['Jurisdiction']

df_jurisdiction_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + jurisdiction_rs).rename(columns={'identifier': ECLI})
df_jurisdiction_li = utils.read_csv(CSV_LI_CASES, [ECLI] + jurisdiction_li)
df_jurisdiction = utils.map_cases(df_jurisdiction_rs.copy(), df_jurisdiction_li.copy())

utils.not_none_rows(df_jurisdiction, number=2)

#### Clean jurisdiction
(remove white spaces before and after text)

In [ ]:
df_jurisdiction_clean = df_jurisdiction.copy()
df_jurisdiction_clean = df_jurisdiction_clean.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))

In [ ]:
df_jurisdiction_clean.describe()


In [ ]:
print('Unique RS spatial values: ', df_jurisdiction_rs['spatial'].unique())
print('Unique RS jurisdiction values: ', df_jurisdiction_rs['jurisdiction_country'].unique())
print('Unique LI jurisdiction values: ', df_jurisdiction_li['Jurisdiction'].unique())

#### Summary jurisdiction

- RS and LI jurisdiction are always "NL" but notation needs to be aligned (LI "Nederland" --> RS "NL")

In [ ]:
del df_jurisdiction, df_jurisdiction_li, df_jurisdiction_rs, df_jurisdiction_clean, jurisdiction_li, jurisdiction_rs

------------------------------------------------------------------------------------------------------------------------
### LINKS
#### Raw links

In [ ]:
links_rs = ['identifier2']
links_li = ['Url', 'OriginalUrl']

df_links_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + links_rs).rename(columns={'identifier': ECLI})
df_links_li = utils.read_csv(CSV_LI_CASES, [ECLI] + links_li)
df_links = utils.map_cases(df_links_rs.copy(), df_links_li.copy())

utils.not_none_rows(df_links, number=2)

#### Clean links
(remove white spaces before and after text)

In [ ]:
df_links_clean = df_links.copy()
df_links_clean = df_links_clean.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))

df_links_clean.describe()

In [ ]:
utils.not_none_rows(df_links_clean, number=5)

#### Summary links:

- RS "identifier2": deeplink to case publication on Rechtspraak.nl
- LI "Url": link to LI entry of case
- LI "OriginalUrl_li": link to case publication of publisher (on Navigator)

In [ ]:
del links_li, links_rs, df_links_clean, df_links, df_links_li, df_links_rs

------------------------------------------------------------------------------------------------------------------------
### SOURCES:
#### Raw sources

In [ ]:
sources_rs = ['source']
sources_li = ['DocumentType']

df_sources_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + sources_rs).rename(columns={'identifier': ECLI})
df_sources_li = utils.read_csv(CSV_LI_CASES, [ECLI] + sources_li)
df_sources = utils.map_cases(df_sources_rs.copy(), df_sources_li.copy())

utils.not_none_rows(df_sources, number=2)

#### Clean sources
(remove white spaces before and after text)

In [ ]:
df_sources_clean = df_sources.copy()
df_sources_clean = df_sources_clean.apply(lambda col: col.apply(
    lambda x: x.strip() if x else x))

df_sources_clean.describe()

In [ ]:
print('Unique RS source values: ', df_sources_rs['source'].unique())
print('Unique LI DocumentType values: ', df_sources_li['DocumentType'].unique())

#### Summary sources:

- RS Source == LI DocumentType == 'Rechtspraak'

In [ ]:
del sources_li, sources_rs, df_sources_clean, df_sources, df_sources_li, df_sources_rs

------------------------------------------------------------------------------------------------------------------------
### ALTERNATIVE SOURCES:
#### Raw alternative sources

In [ ]:
alt_sources_rs = ['hasVersion']
alt_sources_li = ['Sources']

df_alt_sources_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + alt_sources_rs).rename(columns={'identifier': ECLI})
df_alt_sources_li = utils.read_csv(CSV_LI_CASES, [ECLI] + alt_sources_li)
df_alt_sources = utils.map_cases(df_alt_sources_rs.copy(), df_alt_sources_li.copy())

utils.not_none_rows(df_alt_sources, number=2)

#### Clean alt_sources
(remove white spaces before and after text, remove xml tags, format lists)

In [ ]:
df_alt_sources_clean = df_alt_sources.copy()
df_alt_sources_clean = df_alt_sources_clean.apply(lambda col: col.apply(
    lambda x: utils.format_rs_alt_sources(x) if x else x) if col.name in alt_sources_rs else col)
df_alt_sources_clean = df_alt_sources_clean.apply(lambda col: col.apply(
    lambda x: utils.format_li_list(x) if x else x) if col.name in alt_sources_li else col)

df_alt_sources_clean.describe()

In [ ]:
utils.not_none_rows(df_alt_sources_clean, number=5)

#### Summary alternative sources:

- RS hasVersion: list of alternative publications with comment on type of publication
- LI Sources: list of alternative publications
- RS hasVersion and LI Sources do not always match

In [ ]:
del alt_sources_li, alt_sources_rs, df_alt_sources_clean, df_alt_sources_li, df_alt_sources_rs, df_alt_sources

------------------------------------------------------------------------------------------------------------------------
### OTHER:

In [ ]:
other_rs = ['language', 'type', 'procedure'] # 'full_text' too big!

df_other_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + other_rs).rename(columns={'identifier': ECLI})

# clean other (remove white spaces before and after text)
df_other_clean = df_other_rs.apply(lambda col: col.apply(lambda x: x.strip() if x else x))

In [ ]:
df_other_clean.describe()

In [ ]:
utils.not_none_rows(df_other_clean[['procedure']], number=5)

#### Summary other:

- RS language: always 'NL'
- RS type: always 'Uitspraak' (in opinions.csv: always 'Conclusie')
- RS procedure: procedure type of case

In [ ]:
del other_rs, df_other_clean, df_other_rs

------------------------------------------------------------------------------------------------------------------------
###  CASE CITATIONS
#### Raw case citations

In [ ]:
relations_rs = ['relation']
relations_lido = ['target_ecli']
#relations_lido = ['Jurisprudentie'] in new pipeline  # change in new pipeline

df_relations_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + relations_rs).rename(columns={'identifier': ECLI})
df_relations_li = utils.read_csv(CSV_CASE_CITATIONS, ['source_ecli'] + relations_lido).rename(columns={'source_ecli': ECLI})
#df_relations_li = utils.read_csv(CSV_CASE_CITATIONS, [ECLI] + relations_lido)  # change in new pipeline
df_relations = utils.map_cases(df_relations_rs.copy(), df_relations_li.copy())

utils.not_none_rows(df_relations_rs, number=2)

In [ ]:
utils.not_none_rows(df_relations_li, number=2)

#### Clean case citations
(remove white spaces before and after text, format lists)

In [ ]:
df_relations_rs_clean = df_relations_rs.copy()
df_relations_rs_clean = df_relations_rs_clean.apply(lambda col: col.apply(
    lambda x: utils.format_rs_list(x) if x else x) if col.name in relations_rs else col)

df_relations_rs_clean.describe()

In [ ]:
df_relations_li.describe()

In [ ]:
utils.not_none_rows(df_relations_rs_clean, number=5)

In [ ]:
utils.not_none_rows(df_relations, number=5)

#### Summary case citations:

- RS relation: list of case numbers with comment or procedure type (mostly None)
- LIDO target_ecli: ecli of cited case
- no cases with both RS relation and LIDO target_ecli value

In [ ]:
del relations_rs, relations_lido, df_relations_li, df_relations_rs, df_relations

------------------------------------------------------------------------------------------------------------------------
### LEGISLATION CITATIONS
#### Raw legislation citations

In [2]:
references_rs = ['references']
references_lido = ['target_article', 'target_article_webpage']
#references_lido = ['Wet', 'Artikel']  # change in new pipeline


df_references_rs = utils.read_csv(CSV_RS_CASES, ['identifier'] + references_rs).rename(columns={'identifier': ECLI})
df_references_li = utils.read_csv(CSV_LEGISLATION_CITATIONS, ['source_ecli'] + references_lido).rename(columns={'source_ecli': ECLI})
#df_references_li = utils.read_csv(CSV_LEGISLATION_CITATIONS, [ECLI] + references_lido)  # change in new pipeline
df_references = utils.map_cases(df_references_rs.copy(), df_references_li.copy())

,ecli,references,target_article,target_article_webpage
0,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2816734/2016-05-20/2016-05-20,http://wetten.overheid.nl/id/BWBR0005537/2016-05-20/0
1,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2815054/2005-07-01/2005-07-01,http://wetten.overheid.nl/id/BWBR0005537/2005-07-01/0/Hoofdstuk6/Afdeling6.2/Artikel6:8


In [7]:
utils.not_none_rows(df_references, number=500)

,ecli,references,target_article,target_article_webpage
0,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2816734/2016-05-20/2016-05-20,http://wetten.overheid.nl/id/BWBR0005537/2016-05-20/0
1,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2815054/2005-07-01/2005-07-01,http://wetten.overheid.nl/id/BWBR0005537/2005-07-01/0/Hoofdstuk6/Afdeling6.2/Artikel6:8
2,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2816444/2013-01-01/2013-01-01,http://wetten.overheid.nl/id/BWBR0005537/2013-01-01/1/Hoofdstuk8/Titeldeel8.2/Afdeling8.2.6/Artikel8:75
3,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2815134/1994-01-01/1994-01-01,http://wetten.overheid.nl/id/BWBR0005537/2003-03-12/0/Hoofdstuk6/Afdeling6.2/Artikel6:16
4,ECLI:NL:CBB:1996:AH6240,"Algemene wet bestuursrecht, Grondwet, Grondwet 21",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2816734/2016-07-01/2016-07-01,http://wetten.overheid.nl/id/BWBR0005537/2016-07-01/1
...,...,...,...,...
805,ECLI:NL:CBB:1996:ZG0591,"Administratiebesluit Bijzondere Ziektekostenverzekering 6, Wet op de toegang tot ziektekostenverzekeringen 6f",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0002089/2655554/2000-01-01/2000-01-01,http://wetten.overheid.nl/id/BWBR0002089/2006-12-20/0/Artikel8
806,ECLI:NL:CBB:1996:ZG0591,"Administratiebesluit Bijzondere Ziektekostenverzekering 6, Wet op de toegang tot ziektekostenverzekeringen 6f",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2815124/2017-06-12/2017-06-12,http://wetten.overheid.nl/id/BWBR0005537/2017-06-12/0/Hoofdstuk6/Afdeling6.2/Artikel6:15
807,ECLI:NL:CBB:1996:ZG0591,"Administratiebesluit Bijzondere Ziektekostenverzekering 6, Wet op de toegang tot ziektekostenverzekeringen 6f",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0001840/2940704/2008-07-15/2008-07-15,http://wetten.overheid.nl/id/BWBR0001840/2008-07-15/0/Hoofdstuk1/Artikel21
808,ECLI:NL:CBB:1996:ZG0591,"Administratiebesluit Bijzondere Ziektekostenverzekering 6, Wet op de toegang tot ziektekostenverzekeringen 6f",http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2814684/1994-01-01/1994-01-01,http://wetten.overheid.nl/id/BWBR0005537/2002-12-01/0/Hoofdstuk3/Afdeling3.6/Artikel3:45


#### Clean legislation citations
(remove white spaces before and after text, format lists)

In [ ]:
df_references_clean = df_references.copy()
df_references_clean = df_references_clean.apply(lambda col: col.apply(
    lambda x: utils.format_rs_list(x) if x else x) if col.name in references_rs else col)

df_references_clean.describe()

In [ ]:
utils.not_none_rows(df_references_clean, number=5)


### Summary legislation citations:

- RS references: list of legislation titles
- LIDO Wet: link to LIDO entry of legislation or article (linkeddata.overheid.nl)
- LIDO Artikel: link to publication of legislation or article (wetten.overheid.nl)

In [ ]:
del references_lido, references_rs, df_references, df_references_clean, df_references_li, df_references_rs